In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import isnan

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Leitura de dados Carnê

In [4]:
carne = sc.textFile('CARNE.txt')

In [5]:
header = carne.first()
header

'DATA_CONTRATO\tCODIGO_CONTRATO\tCODIGO_CLIENTE\tDATA_VENCIMENTO\tDATA_PAGAMENTO\tVALOR_PARCELA\tVALOR_ENCARGO\tVALOR_VAGO\tNUMERO_PARCELA\tQTD_PAGAMENTOS\tMODALIDADE\tTIPO_PRODUTO'

In [6]:
#filter out the header, make sure the rest looks correct
carne = carne.filter(lambda line: line != header)

In [7]:
#here's where the changes take place
#this creates a dataframe using whatever pyspark feels like using (I think string is the default). the header.split is providing the names of the columns
carne_df = carne.map(lambda k: k.split("\t")).toDF(header.split("\t"))
carne_df.show(5)

+--------------------+---------------+--------------+--------------------+--------------------+-------------+-------------+----------+--------------+--------------+----------+------------+
|       DATA_CONTRATO|CODIGO_CONTRATO|CODIGO_CLIENTE|     DATA_VENCIMENTO|      DATA_PAGAMENTO|VALOR_PARCELA|VALOR_ENCARGO|VALOR_VAGO|NUMERO_PARCELA|QTD_PAGAMENTOS|MODALIDADE|TIPO_PRODUTO|
+--------------------+---------------+--------------+--------------------+--------------------+-------------+-------------+----------+--------------+--------------+----------+------------+
|2018/04/04 00:00:...|   100113272222|            10|2018/05/10 00:00:...|2018/05/07 00:00:...|         78.6|        73.89|      78.6|             1|             1|         5|           1|
|2018/04/04 00:00:...|   100113272222|            10|2018/06/10 00:00:...|2018/06/08 00:00:...|         78.6|        70.04|      78.6|             2|             1|         5|           1|
|2017/02/03 00:00:...|   100081371671|            10|20

# Contagem null CODIGO CLIENTE

In [ ]:
carne_df.filter((carne_df["CODIGO_CLIENTE"] == "") | carne_df["CODIGO_CLIENTE"].isNull() | isnan(carne_df["CODIGO_CLIENTE"])).count()

# Contagem null DATA VENCIMENTO

In [ ]:
carne_df.filter((carne_df["DATA_VENCIMENTO"] == "") | carne_df["DATA_VENCIMENTO"].isNull() | isnan(carne_df["DATA_VENCIMENTO"])).count()

# Contagem null DATA PAGAMENTO

In [ ]:
carne_df.filter((carne_df["DATA_PAGAMENTO"] == "") | carne_df["DATA_PAGAMENTO"].isNull() | isnan(carne_df["DATA_PAGAMENTO"])).count()

In [8]:
carne_df_ts = carne_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
carne_df_ts = carne_df_ts.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))
carne_df_ts = carne_df_ts.withColumn("DATA_CONTRATO", to_timestamp("DATA_CONTRATO", "yyyy/MM/dd HH:mm:ss"))

In [9]:
carne_unique = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE']).dropDuplicates()

In [ ]:
carne_unique.count()

# PAGOU ANTES DO VENCIMENTO

In [10]:
possible_good = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where(carne_df_ts['DATA_PAGAMENTO'] <= carne_df_ts['DATA_VENCIMENTO'])

In [11]:
possible_goodCarne = possible_good.select('CODIGO_CLIENTE').dropDuplicates()


In [ ]:
possible_goodCarne.count()

# PAGOU DEPOIS DO VENCIMENTO

bad_carne = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['CODIGO_CONTRATO'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where(carne_df_ts['DATA_PAGAMENTO'] > carne_df_ts['DATA_VENCIMENTO']) 

In [12]:
bad_carne = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where((carne_df_ts['DATA_PAGAMENTO'] > carne_df_ts['DATA_VENCIMENTO']) | (carne_df_ts["DATA_PAGAMENTO"].isNull()))

In [13]:
bad_clientCarne = bad_carne.select(bad_carne["CODIGO_CLIENTE"]).dropDuplicates()

In [ ]:
bad_clientCarne.count()

# BONS PAGADORES

In [14]:
good_clientCarne = possible_goodCarne.subtract(bad_clientCarne)

In [ ]:
good_clientCarne.count()

# Data Frame Carnê Classificados

In [15]:
tg = good_clientCarne.withColumn('Class', lit(1)).alias('tg')
tb = bad_clientCarne.withColumn('Class', lit(0)).alias('tb')
classificados=tg.union(tb)

In [ ]:
classificados.count()

In [ ]:
classificados.orderBy('CODIGO_CLIENTE').show(10)

In [ ]:
carne_uniquecc = carne_unique.select(carne_unique['CODIGO_CLIENTE'])

In [ ]:
classificados_cc=classificados.select(classificados['CODIGO_CLIENTE'])

In [ ]:
missing = carne_uniquecc.subtract(classificados_cc)
missing.show()
missing.count()

# Cadastro

In [55]:
cadastral = sc.textFile('CADASTRAL.txt', )

In [56]:
header1 = cadastral.first()
header1

'CODIGO_CLIENTE\tDATA_NASCIMENTO\tSEXO\tESTADO_CIVIL\tCIDADE\tUF\tBAIRRO\tCEP\tQTD_DEPENDENTES\tDATA_CADASTRO\tDATA_ADMISSAO\tCATEGORIAL_PROFISSAO\tTIPO_RESIDENCIA\tRENDA_TITULAR\tRENDA_CJ\tOUTRAS_RENDAS\tQTD_CARTOES_ADICIONAIS\tFLAG_CONTA_BANCO\tVALOR_HISTORICO_COMPRAS'

In [57]:
cadastral = cadastral.filter(lambda line: line != header1)

In [58]:
cadastral_df = cadastral.map(lambda k: k.split("\t")).toDF(header1.split("\t"))
cadastral_df.show(5)

+--------------+--------------------+----+------------+-------------+---+--------+--------+---------------+--------------------+--------------------+--------------------+---------------+-------------+--------+-------------+----------------------+----------------+-----------------------+
|CODIGO_CLIENTE|     DATA_NASCIMENTO|SEXO|ESTADO_CIVIL|       CIDADE| UF|  BAIRRO|     CEP|QTD_DEPENDENTES|       DATA_CADASTRO|       DATA_ADMISSAO|CATEGORIAL_PROFISSAO|TIPO_RESIDENCIA|RENDA_TITULAR|RENDA_CJ|OUTRAS_RENDAS|QTD_CARTOES_ADICIONAIS|FLAG_CONTA_BANCO|VALOR_HISTORICO_COMPRAS|
+--------------+--------------------+----+------------+-------------+---+--------+--------+---------------+--------------------+--------------------+--------------------+---------------+-------------+--------+-------------+----------------------+----------------+-----------------------+
|             3|1951/10/21 00:00:...|   F|          CA|   SANTA ROSA| RS|  GLORIA|98900000|              1|1998/10/28 00:00:...|1990/01/

In [65]:
cadastral_df.count()

4277840

In [61]:
cadastral_unique = cadastral_df.select(cadastral_df['CODIGO_CLIENTE']).dropDuplicates()

In [62]:
cadastral_unique.count()

4277840

In [77]:
tabelao.count()

1032081

In [78]:
tabelao.take(5)

[Row(CODIGO_CLIENTE='10000720', DATA_NASCIMENTO='1981/10/27 00:00:00.000000000', SEXO='M', ESTADO_CIVIL='SO', CIDADE='ITAQUI', UF='RS', BAIRRO='ESTACAO', CEP='97650000', QTD_DEPENDENTES='', DATA_CADASTRO='2012/02/16 00:00:00.000000000', DATA_ADMISSAO='', CATEGORIAL_PROFISSAO='AUT+NOMO', TIPO_RESIDENCIA='PR', RENDA_TITULAR='850', RENDA_CJ='', OUTRAS_RENDAS='', QTD_CARTOES_ADICIONAIS='0', FLAG_CONTA_BANCO='', VALOR_HISTORICO_COMPRAS='0.0', Class=0),
 Row(CODIGO_CLIENTE='1000665', DATA_NASCIMENTO='1972/02/02 00:00:00.000000000', SEXO='F', ESTADO_CIVIL='SE', CIDADE='IJUI', UF='RS', BAIRRO='INDUSTRIAL', CEP='98700000', QTD_DEPENDENTES='1', DATA_CADASTRO='2003/09/10 00:00:00.000000000', DATA_ADMISSAO='2002/07/18 00:00:00.000000000', CATEGORIAL_PROFISSAO='ASSALARIADO', TIPO_RESIDENCIA='PR', RENDA_TITULAR='920', RENDA_CJ='', OUTRAS_RENDAS='109', QTD_CARTOES_ADICIONAIS='0', FLAG_CONTA_BANCO='S', VALOR_HISTORICO_COMPRAS='12763.86', Class=0),
 Row(CODIGO_CLIENTE='10007113', DATA_NASCIMENTO='1962/

# Contrato

In [ ]:
contrato = sc.textFile('CONTRATO.txt')

In [ ]:
header2 = contrato.first()
header2

In [ ]:
contrato = contrato.filter(lambda line: line != header2)

In [ ]:
contrato_df = contrato.map(lambda k: k.split("\t")).toDF(header2.split("\t"))
contrato_df.show(5)

In [ ]:
contrato_unique = contrato_df.select(contrato_df['CODIGO_CLIENTE']).dropDuplicates()
contrato_unique.count()

# Cartão

In [16]:
cartao = sc.textFile('CARTAO.txt')

In [17]:
header3 = cartao.first()
header3

'CODIGO_CLIENTE\tCODIGO_CARTAO\tVALOR_MORA\tVALOR_MULTA\tVALOR_TOTAL\tVALOR_MIN\tDATA_VENCIMENTO\tVALOR_PAGO\tTARIFA\tOUTROS_VALORES\tLIMITE\tSOMA_VALOR_DIVIDAS_FUTURAS\tDATA_PAGAMENTO\tQTD_CONTRATOS'

In [18]:
cartao = cartao.filter(lambda line: line != header3)

In [19]:
cartao_df = cartao.map(lambda k: k.split("\t")).toDF(header3.split("\t"))
cartao_df.show(5)

+--------------+-------------+----------+-----------+-----------+---------+--------------------+----------+------+--------------+------+--------------------------+--------------------+-------------+
|CODIGO_CLIENTE|CODIGO_CARTAO|VALOR_MORA|VALOR_MULTA|VALOR_TOTAL|VALOR_MIN|     DATA_VENCIMENTO|VALOR_PAGO|TARIFA|OUTROS_VALORES|LIMITE|SOMA_VALOR_DIVIDAS_FUTURAS|      DATA_PAGAMENTO|QTD_CONTRATOS|
+--------------+-------------+----------+-----------+-----------+---------+--------------------+----------+------+--------------+------+--------------------------+--------------------+-------------+
|            23|    105028440|       0.0|        0.0|     298.79|    64.55|2016/06/10 00:00:...|    298.79|   0.0|          5.99|2604.0|                    8118.6|2016/06/09 00:00:...|            5|
|            23|    105670225|       0.0|        0.0|     307.34|    66.26|2016/07/10 00:00:...|    307.34|   0.0|          5.99|2604.0|                   7811.26|2016/07/08 00:00:...|            6|
|    

In [20]:
cartao_df = cartao_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
cartao_df = cartao_df.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))

# PAGAMENTO ANTES DO VENCIMENTO

In [21]:
possible_goodCartao = cartao_df.select(cartao_df['CODIGO_CLIENTE'],cartao_df['DATA_PAGAMENTO'],cartao_df['DATA_VENCIMENTO']).where(cartao_df['DATA_PAGAMENTO'] <= cartao_df['DATA_VENCIMENTO'])

In [ ]:
possible_goodCartao.count()

In [22]:
pgCartao = possible_goodCartao.select('CODIGO_CLIENTE').dropDuplicates()

In [ ]:
pgCartao.count()

# PAGAMENTO DEPOIS DO VENCIMENTO

In [23]:
bad_cartao = cartao_df.select(cartao_df['CODIGO_CLIENTE'],cartao_df['DATA_PAGAMENTO'],cartao_df['DATA_VENCIMENTO']).where((cartao_df['DATA_PAGAMENTO'] > cartao_df['DATA_VENCIMENTO']) | (cartao_df["DATA_PAGAMENTO"].isNull()))

In [ ]:
bad_cartao.count()

In [24]:
bad_client_cartao = bad_cartao.select('CODIGO_CLIENTE').dropDuplicates()

In [ ]:
bad_client_cartao.count()

# Bons Pagadores Cartão

In [25]:
good_clientCartao= pgCartao.subtract(bad_client_cartao)

In [ ]:
good_clientCartao.count()

In [26]:
cartao_unique = cartao_df.select(cartao_df['CODIGO_CLIENTE']).dropDuplicates()

In [ ]:
cartao_unique.count()

# Tabela Cartao Classificados

In [27]:
tgcard = good_clientCartao.withColumn('Class', lit(1)).alias('tgcard')
tbcard = bad_client_cartao.withColumn('Class', lit(0)).alias('tbcard')
classificadosCartao=tgcard.union(tbcard)

In [ ]:
classificadosCartao.count()

In [ ]:
classificadosCartao.orderBy('CODIGO_CLIENTE').show(10)

# Contagem de NaN

In [ ]:
cartao_df.filter((cartao_df["CODIGO_CLIENTE"] == "") | cartao_df["CODIGO_CLIENTE"].isNull() | isnan(cartao_df["CODIGO_CLIENTE"])).count()

In [ ]:
cartao_df.filter((cartao_df["DATA_VENCIMENTO"] == "") | cartao_df["DATA_VENCIMENTO"].isNull()).count()

In [ ]:
cartao_df.filter((cartao_df["DATA_PAGAMENTO"] == "") | cartao_df["DATA_PAGAMENTO"].isNull()).count()

# Missing Cartão

In [ ]:
cartao_uniquecc = cartao_unique.select(cartao_unique['CODIGO_CLIENTE'])

In [ ]:
classificados_codcliente =classificados.select(classificados['CODIGO_CLIENTE'])

In [ ]:
missing = carne_uniquecc.subtract(classificados_cc)
missing.show()
missing.count()

In [28]:
total_bad = tb.select('CODIGO_CLIENTE').union(tbcard.select('CODIGO_CLIENTE'))

In [29]:
total_bad.show()

+--------------+
|CODIGO_CLIENTE|
+--------------+
|          1090|
|          1512|
|          2069|
|          5325|
|          9586|
|         11888|
|         13772|
|         17401|
|         17686|
|         20428|
|         21248|
|         21452|
|         27317|
|         28839|
|         29454|
|         30002|
|         32558|
|         32812|
|         33174|
|         38986|
+--------------+
only showing top 20 rows



In [40]:
total_bad= tb.select('CODIGO_CLIENTE').union(tbcard.select('CODIGO_CLIENTE')).distinct()

In [37]:
#total_bad.count()

1288463

In [41]:
total_bad.count()

905256

In [33]:
#total_good.count()

215076

In [42]:
total_good = tg.select('CODIGO_CLIENTE').union(tgcard.select('CODIGO_CLIENTE')).distinct()

In [43]:
total_good.count()

177366

In [44]:
og = total_good.subtract(total_bad)

In [45]:
og.count()

126825

In [46]:
og = og.withColumn('Class', lit(1)).alias('og')

In [47]:
total_bad = total_bad.withColumn('Class', lit(0)).alias('total_bad')

In [48]:
classificadosTotal = og.union(total_bad)

In [49]:
classificadosTotal.count()

1032081

In [50]:
classificadosTotal.write.csv('ClassificadosTotal.csv')

In [51]:
ct = classificadosTotal.toPandas()

In [53]:
ct.tail()

,CODIGO_CLIENTE,Class
1032076,12859390,0
1032077,12863303,0
1032078,12865001,0
1032079,12871317,0
1032080,12883967,0


In [54]:
ct.to_csv('ct.csv', header = True)

# TABELÃO

In [76]:
cadastral_df=cadastral_df.alias('cadastral_df')
classificadosTotal=classificadosTotal.alias('classificadosTotal')

tabelao = cadastral_df.join(classificadosTotal, cadastral_df.CODIGO_CLIENTE == classificadosTotal.CODIGO_CLIENTE).select('cadastral_df.*','classificadosTotal.Class')

In [79]:
tabelao_pd=tabelao.toPandas()

In [81]:
tabelao_pd.tail()

,CODIGO_CLIENTE,DATA_NASCIMENTO,SEXO,ESTADO_CIVIL,CIDADE,UF,BAIRRO,CEP,QTD_DEPENDENTES,DATA_CADASTRO,DATA_ADMISSAO,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,RENDA_CJ,OUTRAS_RENDAS,QTD_CARTOES_ADICIONAIS,FLAG_CONTA_BANCO,VALOR_HISTORICO_COMPRAS,Class
1032076,9971734,1983/06/28 00:00:00.000000000,F,SO,TUPANCIRETA,RS,PEDREIRA I,98170000,1,2012/02/13 00:00:00.000000000,2007/02/01 00:00:00.000000000,ASSALARIADO,PR,3000,,,0,,11843.08,0
1032077,9980270,1986/07/14 00:00:00.000000000,M,SO,HORIZONTINA,RS,CENTRO,98920000,0,2012/02/14 00:00:00.000000000,2011/08/01 00:00:00.000000000,ASSALARIADO,AL,"1010,16",,,0,,2901.25,0
1032078,99854,1975/07/19 00:00:00.000000000,F,CO,SANTO ANGELO,RS,MARIO PILAU,98807050,0,1998/12/24 00:00:00.000000000,1997/12/01 00:00:00.000000000,ASSALARIADO,PR,644,,,0,,2402.35,0
1032079,998598,1970/09/17 00:00:00.000000000,F,SO,MONTENEGRO,RS,SANTA RITA,95780000,1,2003/09/04 00:00:00.000000000,2011/02/17 00:00:00.000000000,ASSALARIADO,PR,1121,,,0,,5553.46,0
1032080,999507,1964/03/19 00:00:00.000000000,M,SO,SANTA MARIA,RS,PRESIDENTE JOAO GOULART,97090320,0,2003/09/06 00:00:00.000000000,1998/05/02 00:00:00.000000000,APOSENTADO,PR,2738,,,0,S,103577.96,0


In [85]:
tabelao_pd.loc[tabelao_pd.CODIGO_CLIENTE == '11661811']

,CODIGO_CLIENTE,DATA_NASCIMENTO,SEXO,ESTADO_CIVIL,CIDADE,UF,BAIRRO,CEP,QTD_DEPENDENTES,DATA_CADASTRO,DATA_ADMISSAO,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,RENDA_CJ,OUTRAS_RENDAS,QTD_CARTOES_ADICIONAIS,FLAG_CONTA_BANCO,VALOR_HISTORICO_COMPRAS,Class
1028,11661811,1977/08/08 00:00:00.000000000,F,CA,ITACURUBI,RS,INTERIOR,97685000,1,2014/03/14 00:00:00.000000000,,AUT+NOMO,PR,500,,,0,,5646.49,1


In [86]:
tabelao_pd.to_csv('tabelao.csv', header=True)

In [88]:
len(tabelao_pd.loc[tabelao_pd.Class == 1])

126825

In [93]:
type(tabelao_pd.QTD_DEPENDENTES[0])

str

In [96]:
tabelao_pd.RENDA_TITULAR.astype('int32')

ValueError: invalid literal for int() with base 10: '1899,43'